In [17]:
import pandas as pd 
import numpy as np 
transactions = pd.read_csv("data/transactions.csv")
transactions["day"] = pd.to_datetime(transactions["day"], dayfirst=False)

## Preparing data 
Filter on a single product for now, preparing a dataframe that can be used to fit a demand function

In [18]:
product = "vegetable"

filtered_transactions = transactions[(transactions["category"] == product)]
filtered_transactions["hour"] = pd.to_timedelta(filtered_transactions["time"])

def include_hour(row):
    hour = pd.to_datetime(row["time"]).hour
    return hour
filtered_transactions["hour"] = filtered_transactions.apply(include_hour, axis = 1)
filtered_transactions

<ipython-input-18-148252e6d79b>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-148252e6d79b>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic,hour
8,2018-01-01,9:07:00,0.0,NaN,vegetable,v_21,Boerenkool gesneden,300g,1.29,1.29,0.0,0.0,9.0
12,2018-01-01,9:07:00,0.0,NaN,vegetable,v_32,Buitenbeentjes komkommer,1kg,1.99,1.99,0.0,0.0,9.0
15,2018-01-01,9:50:08,1.0,NaN,vegetable,v_25,Nasi bami Vegetable,450g,1.50,1.50,0.0,0.0,9.0
25,2018-01-01,9:59:31,2.0,72461338.0,vegetable,v_9,Basis worteltjes fijn,680g,0.89,0.89,0.0,1.0,9.0
27,2018-01-01,9:59:31,2.0,72461338.0,vegetable,v_2,Courgette,1st,0.75,0.71,0.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
354850,2018-12-30,20:59:33,20685.0,NaN,vegetable,v_6,Geschrapte worteltjes,500g,1.59,1.59,0.0,0.0,20.0
354853,2018-12-30,20:59:33,20685.0,NaN,vegetable,v_25,Nasi bami Vegetable,450g,1.50,1.50,0.0,0.0,20.0
354856,2018-12-30,20:59:33,20685.0,NaN,vegetable,v_29,Wortelen,500g,1.05,1.05,0.0,0.0,20.0
354858,2018-12-30,20:59:33,20685.0,NaN,vegetable,v_34,SnoepVegetable worteltjes,150g,0.99,0.99,0.0,0.0,20.0


In [19]:
filtered_transactions["discount"] = 100 - (filtered_transactions["purchase_price"]/filtered_transactions["std_sales_price"])*100
filtered_transactions

<ipython-input-19-cb76395263eb>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic,hour,discount
8,2018-01-01,9:07:00,0.0,NaN,vegetable,v_21,Boerenkool gesneden,300g,1.29,1.29,0.0,0.0,9.0,0.000000
12,2018-01-01,9:07:00,0.0,NaN,vegetable,v_32,Buitenbeentjes komkommer,1kg,1.99,1.99,0.0,0.0,9.0,0.000000
15,2018-01-01,9:50:08,1.0,NaN,vegetable,v_25,Nasi bami Vegetable,450g,1.50,1.50,0.0,0.0,9.0,0.000000
25,2018-01-01,9:59:31,2.0,72461338.0,vegetable,v_9,Basis worteltjes fijn,680g,0.89,0.89,0.0,1.0,9.0,0.000000
27,2018-01-01,9:59:31,2.0,72461338.0,vegetable,v_2,Courgette,1st,0.75,0.71,0.0,0.0,9.0,5.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354850,2018-12-30,20:59:33,20685.0,NaN,vegetable,v_6,Geschrapte worteltjes,500g,1.59,1.59,0.0,0.0,20.0,0.000000
354853,2018-12-30,20:59:33,20685.0,NaN,vegetable,v_25,Nasi bami Vegetable,450g,1.50,1.50,0.0,0.0,20.0,0.000000
354856,2018-12-30,20:59:33,20685.0,NaN,vegetable,v_29,Wortelen,500g,1.05,1.05,0.0,0.0,20.0,0.000000
354858,2018-12-30,20:59:33,20685.0,NaN,vegetable,v_34,SnoepVegetable worteltjes,150g,0.99,0.99,0.0,0.0,20.0,0.000000


In [42]:
filtered_transactions["description"].unique()

36

In [52]:
purchases = filtered_transactions.groupby(["description","day"]).describe()
purchases["discount"]

count      mean  std       min       25%  \
description       day                                                    
Basis Maiskorrels 2018-01-01    7.0  5.333333  0.0  5.333333  5.333333   
                  2018-01-02    7.0  0.000000  0.0  0.000000  0.000000   
                  2018-01-03    7.0  0.000000  0.0  0.000000  0.000000   
                  2018-01-04   10.0  0.000000  0.0  0.000000  0.000000   
                  2018-01-05    5.0  0.000000  0.0  0.000000  0.000000   
...                             ...       ...  ...       ...       ...   
Wortelen          2018-12-26    7.0  0.000000  0.0  0.000000  0.000000   
                  2018-12-27    6.0  0.000000  0.0  0.000000  0.000000   
                  2018-12-28    4.0  0.000000  0.0  0.000000  0.000000   
                  2018-12-29    8.0  0.000000  0.0  0.000000  0.000000   
                  2018-12-30   11.0  0.000000  0.0  0.000000  0.000000   

                                   50%       75%       max  
description       day                                       
Basis Maiskorrels 2018-01-01  5.333333  5.333333  5.333333  
                  2018-01-02  0.000000  0.000000  0.000000  
                  2018-01-03  0.000000  0.000000  0.000000  
                  2018-01-04  0.000000  0.000000  0.000000  
                  2018-01-05  0.000000  0.000000  0.000000  
...                                ...       ...       ...  
Wortelen          2018-12-26  0.000000  0.000000  0.000000  
                  2018-12-27  0.000000  0.000000  0.000000  
                  2018-12-28  0.000000  0.000000  0.000000  
                  2018-12-29  0.000000  0.000000  0.000000  
                  2018-12-30  0.000000  0.000000  0.000000  

[12517 rows x 8 columns]

In [53]:
purchases = purchases["discount"]

In [54]:
days = [purchases.index[i][1].dayofyear for i in range(purchases.shape[0])]
products = [purchases.index[i][0] for i in range(purchases.shape[0])]

purchases["product"] = products
purchases["day"] = days
purchases["dayofweek"] = [purchases.index[i][1].dayofweek for i in range(purchases.shape[0])]
pred_cols = ["discount",  "day", "dayofweek"]
target_col = "count"

purchases.rename(columns={"mean":"discount"}, inplace=True)
purchases

count  discount  std       min       25%  \
description       day                                                    
Basis Maiskorrels 2018-01-01    7.0  5.333333  0.0  5.333333  5.333333   
                  2018-01-02    7.0  0.000000  0.0  0.000000  0.000000   
                  2018-01-03    7.0  0.000000  0.0  0.000000  0.000000   
                  2018-01-04   10.0  0.000000  0.0  0.000000  0.000000   
                  2018-01-05    5.0  0.000000  0.0  0.000000  0.000000   
...                             ...       ...  ...       ...       ...   
Wortelen          2018-12-26    7.0  0.000000  0.0  0.000000  0.000000   
                  2018-12-27    6.0  0.000000  0.0  0.000000  0.000000   
                  2018-12-28    4.0  0.000000  0.0  0.000000  0.000000   
                  2018-12-29    8.0  0.000000  0.0  0.000000  0.000000   
                  2018-12-30   11.0  0.000000  0.0  0.000000  0.000000   

                                   50%       75%       max            product  \
description       day                                                           
Basis Maiskorrels 2018-01-01  5.333333  5.333333  5.333333  Basis Maiskorrels   
                  2018-01-02  0.000000  0.000000  0.000000  Basis Maiskorrels   
                  2018-01-03  0.000000  0.000000  0.000000  Basis Maiskorrels   
                  2018-01-04  0.000000  0.000000  0.000000  Basis Maiskorrels   
                  2018-01-05  0.000000  0.000000  0.000000  Basis Maiskorrels   
...                                ...       ...       ...                ...   
Wortelen          2018-12-26  0.000000  0.000000  0.000000           Wortelen   
                  2018-12-27  0.000000  0.000000  0.000000           Wortelen   
                  2018-12-28  0.000000  0.000000  0.000000           Wortelen   
                  2018-12-29  0.000000  0.000000  0.000000           Wortelen   
                  2018-12-30  0.000000  0.000000  0.000000           Wortelen   

                              day  dayofweek  
description       day                         
Basis Maiskorrels 2018-01-01    1          0  
                  2018-01-02    2          1  
                  2018-01-03    3          2  
                  2018-01-04    4          3  
                  2018-01-05    5          4  
...                           ...        ...  
Wortelen          2018-12-26  360          2  
                  2018-12-27  361          3  
                  2018-12-28  362          4  
                  2018-12-29  363          5  
                  2018-12-30  364          6  

[12517 rows x 11 columns]

## Grouped by products and day

In [36]:
import plotly.express as px
#plotting per day
fig = px.scatter(purchases, y="count", x="day", color="discount", color_continuous_scale=px.colors.sequential.Plotly3)
fig.show()

In [16]:
fig = px.scatter(purchases, y="discount", x="count", color="product")
fig.show()

## Implementing a ML model for demand function

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
oh_enc = OneHotEncoder(sparse=False)
oh_enc.fit(purchases[["product"]])
df_oneh_enc = pd.DataFrame(oh_enc.transform(purchases[["product"]]), columns=oh_enc.get_feature_names())
full_purchases = pd.concat([pd.DataFrame(purchases.values, columns=purchases.columns), df_oneh_enc], axis=1)
train, test = train_test_split(full_purchases)
train.dropna(inplace=True)
test.dropna(inplace=True)
X_train = pd.concat([train[pred_cols], train.filter(regex="x0*")], axis=1)
y_train = train[target_col].values

X_test = pd.concat([test[pred_cols], test.filter(regex="x0*")], axis=1).values
y_test = test[target_col].values
X_train

<ipython-input-58-fa437f8c279c>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-58-fa437f8c279c>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,discount,day,dayofweek,max,x0_Basis Maiskorrels,x0_Basis Rode kool met appel,x0_Basis Sperziebonen,x0_Basis erwten zeer fijn,x0_Basis sperziebonen,x0_Basis worteltjes fijn,...,x0_Pompoen stukjes,x0_Rucola,x0_Snijbonen,x0_SnoepVegetable tomaat,x0_SnoepVegetable worteltjes,x0_Spruiten,x0_Verse zuurkool,x0_Witlof,x0_WokVegetable thais,x0_Wortelen
1543,0,98,6,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5491,0,238,6,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
655,0,299,4,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1153,0,71,0,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5349,0,96,4,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10960,35.3535,262,2,35.3535,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5096,0,199,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10311,0,340,3,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
116,5.33333,121,1,5.33333,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
linreg = LinearRegression()
linreg.fit(X_train, y_train)
mae = mean_absolute_error(y_test, linreg.predict(X_test))
mse = mean_squared_error(y_test, linreg.predict(X_test))
print("Mean absolute error: {} Mean Squared error: {}".format(mae, mse))

Mean absolute error: 2.332550672909889 Mean Squared error: 9.170911475715835


In [39]:
pd.DataFrame(X_train, columns=pred_cols)

,discount,day,dayofweek
0,4.891304,288.0,0.0
1,0.000000,296.0,1.0
2,0.000000,82.0,4.0
3,0.000000,61.0,4.0
4,0.000000,350.0,6.0
...,...,...,...
9019,0.000000,264.0,4.0
9020,6.153846,247.0,1.0
9021,0.000000,15.0,0.0
9022,0.000000,120.0,0.0


In [60]:
linreg.coef_

array([ 1.74159463e-01, -5.12605007e-04,  5.29971075e-02,  3.29970079e-02,
       -2.19505583e-01, -4.15073032e-01, -2.43290345e-02, -2.25631172e-02,
        1.44187927e-01, -1.74080562e-01, -3.08363785e+00, -2.49876132e+00,
       -2.92494775e+00, -3.05132003e+00, -2.57159333e+00, -2.66209323e+00,
       -2.76441914e+00, -2.56329725e+00, -3.32693313e+00,  1.72141638e+00,
        1.26741202e+00,  1.11216781e+00,  1.24748440e+00,  1.39948765e+00,
        1.20881968e+00,  1.11062215e+00,  1.74975443e+00,  1.38184340e+00,
        1.21732878e+00,  1.22340602e+00,  1.06132280e+00,  1.50682819e+00,
        1.30311791e+00,  1.03795280e+00,  1.41406123e+00,  1.41687578e+00,
        1.24575328e+00,  1.48276239e+00,  1.04994933e+00])

In [13]:
purchases = filtered_transactions.groupby("day").describe()["purchase_price"]



In [19]:
fig = px.scatter(purchases, x="hour", y="count", color="mean")
fig.show()

In [11]:
purchases[purchases["std"] > 0.001]

count      mean       std   min     25%    50%     75%   max  \
day        hour                                                                 
2018-02-26 9.0    10.0  0.975000  0.217371  0.84  0.8400  0.840  1.1775  1.29   
           10.0    5.0  1.110000  0.246475  0.84  0.8400  1.290  1.2900  1.29   
2018-03-12 9.0     5.0  1.254000  0.032863  1.23  1.2300  1.230  1.2900  1.29   
2018-05-03 9.0     4.0  1.177500  0.225000  0.84  1.1775  1.290  1.2900  1.29   
2018-06-08 9.0     6.0  1.023333  0.130639  0.97  0.9700  0.970  0.9700  1.29   
           10.0    9.0  1.005556  0.106667  0.97  0.9700  0.970  0.9700  1.29   
           19.0    5.0  1.034000  0.143108  0.97  0.9700  0.970  0.9700  1.29   
2018-08-13 9.0     3.0  1.183333  0.184752  0.97  1.1300  1.290  1.2900  1.29   
           10.0    7.0  1.198571  0.156144  0.97  1.1300  1.290  1.2900  1.29   
2018-10-15 9.0     3.0  1.160000  0.225167  0.90  1.0950  1.290  1.2900  1.29   
           11.0    3.0  1.160000  0.225167  0.90  1.0950  1.290  1.2900  1.29   
2018-10-22 10.0    3.0  1.160000  0.225167  0.90  1.0950  1.290  1.2900  1.29   
           11.0    2.0  1.095000  0.275772  0.90  0.9975  1.095  1.1925  1.29   
2018-12-17 11.0    5.0  1.266000  0.032863  1.23  1.2300  1.290  1.2900  1.29   

                 hour  day  dayofweek  
day        hour                        
2018-02-26 9.0    9.0   57          1  
           10.0  10.0   57          1  
2018-03-12 9.0    9.0   71          1  
2018-05-03 9.0    9.0  123          4  
2018-06-08 9.0    9.0  159          5  
           10.0  10.0  159          5  
           19.0  19.0  159          5  
2018-08-13 9.0    9.0  225          1  
           10.0  10.0  225          1  
2018-10-15 9.0    9.0  288          1  
           11.0  11.0  288          1  
2018-10-22 10.0  10.0  295          1  
           11.0  11.0  295          1  
2018-12-17 11.0  11.0  351          1